In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import json
from tqdm import tqdm
from data import load_ff3

In [16]:
# results_max_dir = 'results/models/max'
# results_weighted_dir = 'results/models/weighted'

results_max_dir = 'results/max_L'
results_maxmin_dir = 'results/max-min_LS'
results_weighted_dir = 'results/weighted_LS'

strat_names_max = os.listdir(results_max_dir)
strat_names_maxmin = os.listdir(results_maxmin_dir)
strat_names_weighted = os.listdir(results_weighted_dir)

strat_cum_rets_max = {}
strat_cum_rets_maxmin = {}
strat_cum_rets_weighted = {}

strat_metrics_max = {}
strat_metrics_maxmin = {}
strat_metrics_weighted = {}

for strat_name in strat_names_max:
    strat_cum_rets_max[strat_name] = pd.read_csv(os.path.join(results_max_dir, strat_name, 'cumulative_returns.csv'), index_col=0)
    strat_metrics_max[strat_name] = json.load(open(os.path.join(results_max_dir, strat_name, 'params.json'), 'r'))
    
for strat_name in strat_names_maxmin:
    strat_cum_rets_maxmin[strat_name] = pd.read_csv(os.path.join(results_maxmin_dir, strat_name, 'cumulative_returns.csv'), index_col=0)
    strat_metrics_maxmin[strat_name] = json.load(open(os.path.join(results_maxmin_dir, strat_name, 'params.json'), 'r'))
    
for strat_name in strat_names_weighted:
    strat_cum_rets_weighted[strat_name] = pd.read_csv(os.path.join(results_weighted_dir, strat_name, 'cumulative_returns.csv'), index_col=0)
    strat_metrics_weighted[strat_name] = json.load(open(os.path.join(results_weighted_dir, strat_name, 'params.json'), 'r'))
    
    
# Sort each dict by dict name
strat_cum_rets_max = dict(sorted(strat_cum_rets_max.items(), key=lambda x: len(x[0])))
strat_cum_rets_maxmin = dict(sorted(strat_cum_rets_maxmin.items(), key=lambda x: len(x[0])))
strat_cum_rets_weighted = dict(sorted(strat_cum_rets_weighted.items(), key=lambda x: len(x[0])))

strat_metrics_max = dict(sorted(strat_metrics_max.items(), key=lambda x: len(x[0])))
strat_metrics_maxmin = dict(sorted(strat_metrics_maxmin.items(), key=lambda x: len(x[0])))
strat_metrics_weighted = dict(sorted(strat_metrics_weighted.items(), key=lambda x: len(x[0])))

len(strat_cum_rets_max), len(strat_cum_rets_weighted), len(strat_metrics_max), len(strat_metrics_weighted)

In [17]:
def calc_sharpe(cum_rets, rf_ret):
    # Get monthly returns
    daily_rets = cum_rets.pct_change().dropna()
    daily_rets.index = pd.to_datetime(daily_rets.index, format='%Y-%m-%d')
    
    merge = pd.merge(daily_rets, rf_ret, left_index=True, right_index=True)

    # Calculate sharpe ratio
    sharpe = ((merge['0']*100 - merge['RF']) / (merge['0']*100).std()).mean()
    return sharpe

ff3_daily = load_ff3()
ff3_daily = ff3_daily[ff3_daily.index >= 20180305]
ff3_daily.index = pd.to_datetime(ff3_daily.index, format='%Y%m%d')

for strat_name, metric in strat_metrics_max.items():
    sharpe = calc_sharpe(strat_cum_rets_max[strat_name], ff3_daily['RF'])
    metric['sharpe'] = sharpe

for strat_name, metric in strat_metrics_maxmin.items():
    sharpe = calc_sharpe(strat_cum_rets_maxmin[strat_name], ff3_daily['RF'])
    metric['sharpe'] = sharpe

for strat_name, metric in strat_metrics_weighted.items():
    sharpe = calc_sharpe(strat_cum_rets_weighted[strat_name], ff3_daily['RF'])
    metric['sharpe'] = sharpe

In [18]:
ff3_daily = pd.read_csv('ff3_daily.csv', index_col=0)

In [19]:
def plot_cum_rets(cum_rets: dict[str, pd.DataFrame], title: str, figsize=(10, 6), ff3_daily=ff3_daily):
    if len(cum_rets) == 0:
        return
    
    fig, ax = plt.subplots(figsize=figsize)
    for strat_name in cum_rets.keys():
        rets = cum_rets[strat_name]
        rets.index = pd.to_datetime(rets.index)
        rets_monthly = rets.resample('ME').last()
        
        first_date = int(f"{rets_monthly.index[0].year}{'0' if rets_monthly.index[0].month < 10 else ''}{rets_monthly.index[0].month}{'0' if rets_monthly.index[0].day < 10 else ''}{rets_monthly.index[0].day}")
        
        ax.plot(rets_monthly.index, rets_monthly.values, label=strat_name)

    ff3_daily.index = pd.to_datetime(ff3_daily.index, format='%Y%m%d')
    ff3_daily = ff3_daily[ff3_daily.index > pd.to_datetime(first_date, format='%Y%m%d')]
    mkt_cum_rets_daily = (1 + (ff3_daily['Mkt-RF'] + ff3_daily['RF']) / 100).cumprod()
    mkt_cum_rets_monthly = mkt_cum_rets_daily.resample('ME').last()
    ax.plot(mkt_cum_rets_monthly.index, mkt_cum_rets_monthly.values, label='Market')
    
    ax.legend()
    plt.xlabel('Date')
    plt.ylabel('Cumulative Returns')
    plt.title(title)
    plt.show()

In [20]:
lstm_industries_max_equal_5 = {}
lstm_industries_max_equal_metrics_5 = {}
for name, rets in strat_cum_rets_max.items():
    if 'lstm' in name.lower() and 'industries' in name.lower() and 'equal' in name.lower() and '38' not in name.lower() and '5days' in name.lower():
        lstm_industries_max_equal_5[name] = rets
        lstm_industries_max_equal_metrics_5[name] = strat_metrics_max[name]
        
lstm_industries_max_value_5 = {}
lstm_industries_max_value_metrics_5 = {}
for name, rets in strat_cum_rets_max.items():
    if 'lstm' in name.lower() and 'industries' in name.lower() and 'value' in name.lower() and '38' not in name.lower() and '5days' in name.lower():
        lstm_industries_max_value_5[name] = rets
        lstm_industries_max_value_metrics_5[name] = strat_metrics_max[name]
        
lstm_industries_maxmin_equal_5 = {}
lstm_industries_maxmin_equal_metrics_5 = {}
for name, rets in strat_cum_rets_maxmin.items():
    if 'lstm' in name.lower() and 'industries' in name.lower() and 'equal' in name.lower() and '38' not in name.lower() and '5days' in name.lower():
        lstm_industries_maxmin_equal_5[name] = rets
        lstm_industries_maxmin_equal_metrics_5[name] = strat_metrics_maxmin[name]
        
lstm_industries_maxmin_value_5 = {}
lstm_industries_maxmin_value_metrics_5 = {}
for name, rets in strat_cum_rets_maxmin.items():
    if 'lstm' in name.lower() and 'industries' in name.lower() and 'value' in name.lower() and '38' not in name.lower() and '5days' in name.lower():
        lstm_industries_maxmin_value_5[name] = rets
        lstm_industries_maxmin_value_metrics_5[name] = strat_metrics_maxmin[name]

lstm_industries_weighted_equal_5 = {}
lstm_industries_weighted_equal_metrics_5 = {}
for name, rets in strat_cum_rets_weighted.items():
    if 'lstm' in name.lower() and 'industries' in name.lower() and 'equal' in name.lower() and '38' not in name.lower() and '5days' in name.lower():
        lstm_industries_weighted_equal_5[name] = rets
        lstm_industries_weighted_equal_metrics_5[name] = strat_metrics_weighted[name]
                
lstm_industries_weighted_value_5 = {}
lstm_industries_weighted_value_metrics_5 = {}
for name, rets in strat_cum_rets_weighted.items():
    if 'lstm' in name.lower() and 'industries' in name.lower() and 'value' in name.lower() and '38' not in name.lower() and '5days' in name.lower():
        lstm_industries_weighted_value_5[name] = rets
        lstm_industries_weighted_value_metrics_5[name] = strat_metrics_weighted[name]

# plot_cum_rets(lstm_industries_max_equal_5, 'LSTM Industries, 5 Day, Trade Max Return, From Equal-Weighted Data')
# plot_cum_rets(lstm_industries_max_value_5, 'LSTM Industries, 5 Day, Trade Max Return, From Value-Weighted Data')

plot_cum_rets(lstm_industries_maxmin_equal_5, 'LSTM Industries, 5 Day, Trade Max-Min Return, From Equal-Weighted Data')
plot_cum_rets(lstm_industries_maxmin_value_5, 'LSTM Industries, 5 Day, Trade Max-Min Return, From Value-Weighted Data')

plot_cum_rets(lstm_industries_weighted_equal_5, 'LSTM Industries, 5 Day, Trade Weighted Returns, From Equal-Weighted Data')
plot_cum_rets(lstm_industries_weighted_value_5, 'LSTM Industries, 5 Day, Trade Weighted Returns, From Value-Weighted Data')

In [21]:
# Same but _22
lstm_industries_max_equal_22 = {}
lstm_industries_max_equal_metrics_22 = {}
for name, rets in strat_cum_rets_max.items():
    if 'lstm' in name.lower() and 'industries' in name.lower() and 'equal' in name.lower() and '38' not in name.lower() and '22days' in name.lower():
        lstm_industries_max_equal_22[name] = rets
        lstm_industries_max_equal_metrics_22[name] = strat_metrics_max[name]
        
lstm_industries_max_value_22 = {}
lstm_industries_max_value_metrics_22 = {}
for name, rets in strat_cum_rets_max.items():
    if 'lstm' in name.lower() and 'industries' in name.lower() and 'value' in name.lower() and '38' not in name.lower() and '22days' in name.lower():
        lstm_industries_max_value_22[name] = rets
        lstm_industries_max_value_metrics_22[name] = strat_metrics_max[name]
        
lstm_industries_weighted_equal_22 = {}
lstm_industries_weighted_equal_metrics_22 = {}
for name, rets in strat_cum_rets_weighted.items():
    if 'lstm' in name.lower() and 'industries' in name.lower() and 'equal' in name.lower() and '38' not in name.lower() and '22days' in name.lower():
        lstm_industries_weighted_equal_22[name] = rets
        lstm_industries_weighted_equal_metrics_22[name] = strat_metrics_weighted[name]
        
lstm_industries_weighted_value_22 = {}
lstm_industries_weighted_value_metrics_22 = {}
for name, rets in strat_cum_rets_weighted.items():
    if 'lstm' in name.lower() and 'industries' in name.lower() and 'value' in name.lower() and '38' not in name.lower() and '22days' in name.lower():
        lstm_industries_weighted_value_22[name] = rets
        lstm_industries_weighted_value_metrics_22[name] = strat_metrics_weighted[name]
        
plot_cum_rets(lstm_industries_max_equal_22, 'LSTM Industries, 22 Day, Trade Max Return, From Equal-Weighted Data')
plot_cum_rets(lstm_industries_max_value_22, 'LSTM Industries, 22 Day, Trade Max Return, From Value-Weighted Data')
plot_cum_rets(lstm_industries_weighted_equal_22, 'LSTM Industries, 22 Day, Trade Weighted Returns, From Equal-Weighted Data')
plot_cum_rets(lstm_industries_weighted_value_22, 'LSTM Industries, 22 Day, Trade Weighted Returns, From Value-Weighted Data')

In [22]:
XGBRegressor_industries_max_equal_5 = {}
XGBRegressor_industries_max_equal_metrics_5 = {}
for name, rets in strat_cum_rets_max.items():
    if 'XGBRegressor' in name and 'industries' in name.lower() and 'equal' in name.lower() and '38' not in name.lower() and '5days' in name.lower():
        XGBRegressor_industries_max_equal_5[name] = rets
        XGBRegressor_industries_max_equal_metrics_5[name] = strat_metrics_max[name]

XGBRegressor_industries_max_value_5 = {}
XGBRegressor_industries_max_value_metrics_5 = {}
for name, rets in strat_cum_rets_max.items():
    if 'XGBRegressor' in name and 'industries' in name.lower() and 'value' in name.lower() and '38' not in name.lower() and '5days' in name.lower():
        XGBRegressor_industries_max_value_5[name] = rets
        XGBRegressor_industries_max_value_metrics_5[name] = strat_metrics_max[name]

XGBRegressor_industries_weighted_equal_5 = {}
XGBRegressor_industries_weighted_equal_metrics_5 = {}
for name, rets in strat_cum_rets_weighted.items():
    if 'XGBRegressor' in name and 'industries' in name.lower() and 'equal' in name.lower() and '38' not in name.lower() and '5days' in name.lower():
        XGBRegressor_industries_weighted_equal_5[name] = rets
        XGBRegressor_industries_weighted_equal_metrics_5[name] = strat_metrics_weighted[name]

XGBRegressor_industries_weighted_value_5 = {}
XGBRegressor_industries_weighted_value_metrics_5 = {}
for name, rets in strat_cum_rets_weighted.items():
    if 'XGBRegressor' in name and 'industries' in name.lower() and 'value' in name.lower() and '38' not in name.lower() and '5days' in name.lower():
        XGBRegressor_industries_weighted_value_5[name] = rets
        XGBRegressor_industries_weighted_value_metrics_5[name] = strat_metrics_weighted[name]

plot_cum_rets(XGBRegressor_industries_max_equal_5, 'XGBoost Industries, 5 Day, Trade Max Return, From Equal-Weighted Data')
plot_cum_rets(XGBRegressor_industries_max_value_5, 'XGBoost Industries, 5 Day, Trade Max Return, From Value-Weighted Data')
plot_cum_rets(XGBRegressor_industries_weighted_equal_5, 'XGBoost Industries, 5 Day, Trade Weighted Returns, From Equal-Weighted Data')
plot_cum_rets(XGBRegressor_industries_weighted_value_5, 'XGBoost Industries, 5 Day, Trade Weighted Returns, From Value-Weighted Data')

In [23]:
# Do the same but for 22 days
XGBRegressor_industries_max_equal_22 = {}
XGBRegressor_industries_max_equal_metrics_22 = {}
for name, rets in strat_cum_rets_max.items():
    if 'XGBRegressor' in name and 'industries' in name.lower() and 'equal' in name.lower() and '38' not in name.lower() and '22days' in name.lower():
        XGBRegressor_industries_max_equal_22[name] = rets
        XGBRegressor_industries_max_equal_metrics_22[name] = strat_metrics_max[name]
        
XGBRegressor_industries_max_value_22 = {}
XGBRegressor_industries_max_value_metrics_22 = {}
for name, rets in strat_cum_rets_max.items():
    if 'XGBRegressor' in name and 'industries' in name.lower() and 'value' in name.lower() and '38' not in name.lower() and '22days' in name.lower():
        XGBRegressor_industries_max_value_22[name] = rets
        XGBRegressor_industries_max_value_metrics_22[name] = strat_metrics_max[name]
        
XGBRegressor_industries_weighted_equal_22 = {}
XGBRegressor_industries_weighted_equal_metrics_22 = {}
for name, rets in strat_cum_rets_weighted.items():
    if 'XGBRegressor' in name and 'industries' in name.lower() and 'equal' in name.lower() and '38' not in name.lower() and '22days' in name.lower():
        XGBRegressor_industries_weighted_equal_22[name] = rets
        XGBRegressor_industries_weighted_equal_metrics_22[name] = strat_metrics_weighted[name]
        
XGBRegressor_industries_weighted_value_22 = {}
XGBRegressor_industries_weighted_value_metrics_22 = {}
for name, rets in strat_cum_rets_weighted.items():
    if 'XGBRegressor' in name and 'industries' in name.lower() and 'value' in name.lower() and '38' not in name.lower() and '22days' in name.lower():
        XGBRegressor_industries_weighted_value_22[name] = rets
        XGBRegressor_industries_weighted_value_metrics_22[name] = strat_metrics_weighted[name] 
        
plot_cum_rets(XGBRegressor_industries_max_equal_22, 'XGBoost Industries, 22 Day, Trade Max Return, From Equal-Weighted Data')
plot_cum_rets(XGBRegressor_industries_max_value_22, 'XGBoost Industries, 22 Day, Trade Max Return, From Value-Weighted Data')
plot_cum_rets(XGBRegressor_industries_weighted_equal_22, 'XGBoost Industries, 22 Day, Trade Weighted Returns, From Equal-Weighted Data')
plot_cum_rets(XGBRegressor_industries_weighted_value_22, 'XGBoost Industries, 22 Day, Trade Weighted Returns, From Value-Weighted Data')

In [24]:
ff6_max_5 = {}
ff6_max_metrics_5 = {}
for name, rets in strat_cum_rets_max.items():
    if 'ff6' in name.lower() and '5days' in name.lower():
        ff6_max_5[name] = rets
        ff6_max_metrics_5[name] = strat_metrics_max[name]

ff6_weighted_5 = {}
ff6_weighted_metrics_5 = {}
for name, rets in strat_cum_rets_weighted.items():
    if 'ff6' in name.lower() and '5days' in name.lower():
        ff6_weighted_5[name] = rets
        ff6_weighted_metrics_5[name] = strat_metrics_weighted[name]
   
ff6_max_22 = {}
ff6_max_metrics_22 = {}
for name, rets in strat_cum_rets_max.items():
    if 'ff6' in name.lower() and '22days' in name.lower():
        ff6_max_22[name] = rets
        ff6_max_metrics_22[name] = strat_metrics_max[name]
   
ff6_weighted_22 = {}
ff6_weighted_metrics_22 = {}
for name, rets in strat_cum_rets_weighted.items():
    if 'ff6' in name.lower() and '22days' in name.lower():
        ff6_weighted_22[name] = rets
        ff6_weighted_metrics_22[name] = strat_metrics_weighted[name]
           
        
plot_cum_rets(ff6_max_5, 'FF6, 5 Day, Trade Max Return')
plot_cum_rets(ff6_weighted_5, 'FF6, 5 Day, Trade Weighted Returns')
plot_cum_rets(ff6_max_22, 'FF6, 22 Day, Trade Max Return')
plot_cum_rets(ff6_weighted_22, 'FF6, 22 Day, Trade Weighted Returns')

In [25]:
ff6_max_22 = {}
ff6_max_metrics_22 = {}
for name, rets in strat_cum_rets_max.items():
    if 'ff6' in name.lower() and '22days' in name.lower():
        ff6_max_22[name] = rets
        ff6_max_metrics_22[name] = strat_metrics_max[name]
        
ff6_weighted_22 = {}
ff6_weighted_metrics_22 = {}
for name, rets in strat_cum_rets_weighted.items():
    if 'ff6' in name.lower() and '22days' in name.lower():
        ff6_weighted_22[name] = rets
        ff6_weighted_metrics_22[name] = strat_metrics_weighted[name]
        
plot_cum_rets(ff6_max_22, 'FF6, 22 Day, Trade Max Return')
plot_cum_rets(ff6_weighted_22, 'FF6, 22 Day, Trade Weighted Returns')

In [26]:
# Make df for each metrics dict
df_metrics_dict = {}

df_metrics_dict['lstm_industries_max_equal_metrics_5'] = pd.DataFrame(lstm_industries_max_equal_metrics_5).T
df_metrics_dict['lstm_industries_max_value_metrics_5'] = pd.DataFrame(lstm_industries_max_value_metrics_5).T
df_metrics_dict['lstm_industries_weighted_equal_metrics_5'] = pd.DataFrame(lstm_industries_weighted_equal_metrics_5).T
df_metrics_dict['lstm_industries_weighted_value_metrics_5'] = pd.DataFrame(lstm_industries_weighted_value_metrics_5).T

df_metrics_dict['lstm_industries_max_equal_metrics_22'] = pd.DataFrame(lstm_industries_max_equal_metrics_22).T
df_metrics_dict['lstm_industries_max_value_metrics_22'] = pd.DataFrame(lstm_industries_max_value_metrics_22).T
df_metrics_dict['lstm_industries_weighted_equal_metrics_22'] = pd.DataFrame(lstm_industries_weighted_equal_metrics_22).T
df_metrics_dict['lstm_industries_weighted_value_metrics_22'] = pd.DataFrame(lstm_industries_weighted_value_metrics_22).T

df_metrics_dict['XGBRegressor_industries_max_equal_metrics_5'] = pd.DataFrame(XGBRegressor_industries_max_equal_metrics_5).T
df_metrics_dict['XGBRegressor_industries_max_value_metrics_5'] = pd.DataFrame(XGBRegressor_industries_max_value_metrics_5).T
df_metrics_dict['XGBRegressor_industries_weighted_equal_metrics_5'] = pd.DataFrame(XGBRegressor_industries_weighted_equal_metrics_5).T
df_metrics_dict['XGBRegressor_industries_weighted_value_metrics_5'] = pd.DataFrame(XGBRegressor_industries_weighted_value_metrics_5).T

df_metrics_dict['XGBRegressor_industries_max_equal_metrics_22'] = pd.DataFrame(XGBRegressor_industries_max_equal_metrics_22).T
df_metrics_dict['XGBRegressor_industries_max_value_metrics_22'] = pd.DataFrame(XGBRegressor_industries_max_value_metrics_22).T
df_metrics_dict['XGBRegressor_industries_weighted_equal_metrics_22'] = pd.DataFrame(XGBRegressor_industries_weighted_equal_metrics_22).T
df_metrics_dict['XGBRegressor_industries_weighted_value_metrics_22'] = pd.DataFrame(XGBRegressor_industries_weighted_value_metrics_22).T

df_metrics_dict['ff6_max_metrics_5'] = pd.DataFrame(ff6_max_metrics_5).T
df_metrics_dict['ff6_weighted_metrics_5'] = pd.DataFrame(ff6_weighted_metrics_5).T

df_metrics_dict['ff6_max_metrics_22'] = pd.DataFrame(ff6_max_metrics_22).T
df_metrics_dict['ff6_weighted_metrics_22'] = pd.DataFrame(ff6_weighted_metrics_22).T


def highlight(data, color='yellow'):
    attr = 'font-weight: bold'
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_max = data == data.max()
        return [attr if v else '' for v in is_max]
    else:  # from .apply(axis=None)
        is_max = data == data.max().max()
        return pd.DataFrame(np.where(is_max, attr, ''),
                            index=data.index, columns=data.columns)

# Specify the columns you want to highlight
columns_to_highlight = ['alpha']

for df_name, df in df_metrics_dict.items():
    # If df is empty, skip
    if df.empty:
        continue
    styled_df = df.style.apply(highlight, subset=columns_to_highlight)
    print(df_name)
    display(styled_df)

In [27]:
cols = ['alpha', 'alpha_annual', 'alpha_p_value', 'beta', 'sharpe']
metrics_df = pd.DataFrame(columns=cols)

lstm_best_5 = {}
lstm_best_22 = {}
lstm_best_5_metrics = metrics_df.copy()
lstm_best_22_metrics = metrics_df.copy()

XGBRegressor_best_5 = {}
XGBRegressor_best_22 = {}
XGBRegressor_best_5_metrics = metrics_df.copy()
XGBRegressor_best_22_metrics = metrics_df.copy()

lstm_best_ff6 = {}
XGBRegressor_best_ff6 = {}
lstm_best_ff6_metrics = metrics_df.copy()
XGBRegressor_best_ff6_metrics = metrics_df.copy()

for df_name, df in df_metrics_dict.items():
    if df.empty:
        continue
    
    # Find rows with alpha_p_value < 0.05
    significant_alpha_df = df[df['alpha_p_value'] < 0.05]
    
    if 'ff6' in df_name:
        for strat_name in df.index:
            if 'max' in df_name:
                strat_fullname = '_'.join(strat_name.split('_')[:-1] + ['max'] + strat_name.split('_')[-1:])
                
                if 'LSTM' in strat_name:
                    lstm_best_ff6[strat_fullname] = strat_cum_rets_max[strat_name]
                    lstm_best_ff6_metrics.loc[strat_fullname] = df.loc[strat_name]
                elif 'XGBRegressor' in strat_name:
                    XGBRegressor_best_ff6[strat_fullname] = strat_cum_rets_max[strat_name]
                    XGBRegressor_best_ff6_metrics.loc[strat_fullname] = df.loc[strat_name]
                    
            elif 'weighted' in df_name:
                strat_fullname = '_'.join(strat_name.split('_')[:-1] + ['weighted'] + strat_name.split('_')[-1:])
                
                if 'LSTM' in strat_name:
                    lstm_best_ff6[strat_fullname] = strat_cum_rets_weighted[strat_name]
                    lstm_best_ff6_metrics.loc[strat_fullname] = df.loc[strat_name]
                elif 'XGBRegressor' in strat_name:
                    XGBRegressor_best_ff6[strat_fullname] = strat_cum_rets_weighted[strat_name]
                    XGBRegressor_best_ff6_metrics.loc[strat_fullname] = df.loc[strat_name]
                    
        
    else:
        if len(significant_alpha_df) > 0:
            best_strat = significant_alpha_df['alpha'].idxmax()
        else:
            best_strat = df['alpha_p_value'].idxmin()
        if 'max' in df_name:
            best_strat_fullname =  '_'.join(best_strat.split('_')[:-1] + ['max'] + best_strat.split('_')[-1:])
            
            if '5' in df_name and 'lstm' in df_name:
                lstm_best_5[best_strat_fullname] = strat_cum_rets_max[best_strat]
                lstm_best_5_metrics.loc[best_strat_fullname] = df.loc[best_strat]
            elif '22' in df_name and 'lstm' in df_name:
                lstm_best_22[best_strat_fullname] = strat_cum_rets_max[best_strat]
                lstm_best_22_metrics.loc[best_strat_fullname] = df.loc[best_strat]
            elif '5' in df_name and 'XGBRegressor' in df_name:
                XGBRegressor_best_5[best_strat_fullname] = strat_cum_rets_max[best_strat]
                XGBRegressor_best_5_metrics.loc[best_strat_fullname] = df.loc[best_strat]
            elif '22' in df_name and 'XGBRegressor' in df_name:
                XGBRegressor_best_22[best_strat_fullname] = strat_cum_rets_max[best_strat]
                XGBRegressor_best_22_metrics.loc[best_strat_fullname] = df.loc[best_strat]
            
        elif 'weighted' in df_name:
            best_strat_fullname = '_'.join(best_strat.split('_')[:-1] + ['weighted'] + best_strat.split('_')[-1:])
        
            if '5' in df_name and 'lstm' in df_name:
                lstm_best_5[best_strat_fullname] = strat_cum_rets_weighted[best_strat]
                lstm_best_5_metrics.loc[best_strat_fullname] = df.loc[best_strat]
            elif '22' in df_name and 'lstm' in df_name:
                lstm_best_22[best_strat_fullname] = strat_cum_rets_weighted[best_strat]
                lstm_best_22_metrics.loc[best_strat_fullname] = df.loc[best_strat]
            elif '5' in df_name and 'XGBRegressor' in df_name:
                XGBRegressor_best_5[best_strat_fullname] = strat_cum_rets_weighted[best_strat]
                XGBRegressor_best_5_metrics.loc[best_strat_fullname] = df.loc[best_strat]
            elif '22' in df_name and 'XGBRegressor' in df_name:
                XGBRegressor_best_22[best_strat_fullname] = strat_cum_rets_weighted[best_strat]
                XGBRegressor_best_22_metrics.loc[best_strat_fullname] = df.loc[best_strat]
                
    

In [28]:
plot_cum_rets(lstm_best_5, 'Best LSTM Industry Strategies, 5 Day')
# display but all numbers rounded to 3 decimal places
display(lstm_best_5_metrics.round(3))

plot_cum_rets(lstm_best_22, 'Best LSTM Industry Strategies, 22 Day')
display(lstm_best_22_metrics.round(3))

plot_cum_rets(XGBRegressor_best_5, 'Best Gradient Boosting Industry Strategies, 5 Day')
display(XGBRegressor_best_5_metrics.round(3))

plot_cum_rets(XGBRegressor_best_22, 'Best Gradient Boosting Industry Strategies, 22 Day')
display(XGBRegressor_best_22_metrics.round(3))

plot_cum_rets(lstm_best_ff6, 'LSTM FF6 Strategies') 
display(lstm_best_ff6_metrics.round(3))

plot_cum_rets(XGBRegressor_best_ff6, 'Gradient Boosting FF6 Strategies')
display(XGBRegressor_best_ff6_metrics.round(3))